# BikeMaps.Org Data for Victoria, BC

# Abstract

# Introduction 

# Methods

# Results

# Discussion

# Conclusion

## Data Exploration

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import numpy as np
import pysal
import pysal.spatial_dynamics.interaction as interaction
np.random.seed(100)
import datetime

In [185]:
collisions = pd.read_csv('/Users/marthamorrissey/Documents/STA_project_MM/Data/col.csv')
collisions.dtypes

#collisions["t"] = collisions.date.apply(datetime.datetime.strptime, '%Y-%m-%d %H:%M:%S')
#collisions["time"] = collisions.date.apply(datetime.date.isoformat)

i_type               object
incident_with        object
date                 object
p_type               object
details              object
incident_effect      object
injury               object
trip_purpose         object
regular_cyclist      object
helmet               object
intoxicated          object
road_conditions      object
sightlines           object
cars_on_roadside     object
riding_on            object
bike_lights          object
terrain              object
direction            object
turning              object
age                 float64
birthmonth          float64
sex                  object
pk                    int64
x                   float64
y                   float64
dtype: object

In [3]:
theft = pd.read_csv('/Users/marthamorrissey/Documents/STA_project_MM/Data/theft.csv')
theft.head()

,i_type,date,p_type,details,how_locked,lock,locked_to,lighting,traffic,police_report,police_report_num,insurance_claim,insurance_claim_num,regular_cyclist,pk,longitude,latitude
0,Minor bike component,2015-06-29T19:30:00,theft,Quick release removed from back wheel - i.e. i...,Frame and tire locked,U-Lock,Good,NaN,Medium,True,T15000455,False,NaN,Y,1253,-123.386807,48.437529
1,Bike (value < $1000),2015-07-28T02:00:00,theft,Bike stolen from under the Petch Building bree...,Frame locked,Cable lock,Moderate,NaN,Very Low,True,NaN,False,NaN,Y,1421,-123.310697,48.461660
2,Minor bike component,2015-07-24T21:30:00,theft,Bike helmet stolen. It was not locked.,Frame and tire locked,Cable lock,Moderate,NaN,High,False,NaN,False,NaN,Y,1493,-123.364642,48.427851
3,Bike (value < $1000),2015-04-17T21:00:00,theft,parked at work inside locked bike cage room. D...,Not locked,NaN,Moderate,NaN,Very Low,True,NaN,False,NaN,Y,1614,-123.361670,48.426880
4,Bike (value < $1000),2015-02-03T17:30:00,theft,"This was an unfortunate situation where I ""loc...",Not locked,NaN,Good,NaN,Low,True,NaN,False,NaN,Y,1636,-123.378739,48.440147


In [34]:
#hazards = pd.read_csv('/Users/marthamorrissey/Documents/STA_project_MM/Data/hazards.csv')

In [4]:
near_miss = pd.read_csv('/Users/marthamorrissey/Documents/STA_project_MM/Data/near_miss.csv')

In [36]:
#collisions.dtypes

In [197]:
#collisions['T'] = pd.to_datetime(collisions['date'], format='%Y-%m-%dT%H:%M:%S')

#collisions.head()


#pd.to_datetime('2014-12-04 22:30:00')

#pd.Timestamp(np.datetime64('2014-12-04 22:30:00'))


date = collisions['date']


def changetime(x):
    t1 = pd.to_datetime(x, format ='%Y-%m-%dT%H:%M:%S')
    #t2 = pd.Timestamp(np.datetime64(t1))
    t3 = t1.to_pydatetime()
    return t3

                               
collisions['T_test3'] = collisions['date'].apply(changetime)

collisions['T_test3']

#collisions.dtypes

#y = changetime('2017-02-03T17:30:00')

#y

#type(y)

collisions['just_date'] = collisions['T_test3'].dt.date

collisions.head()


,i_type,incident_with,date,p_type,details,incident_effect,injury,trip_purpose,regular_cyclist,helmet,...,turning,age,birthmonth,sex,pk,x,y,T_test3,timee,just_date
0,Collision with moving object or vehicle,"Vehicle, angle",2014-12-04T22:30:00,collision,Cab pulled an illegal u-turn and I collided wi...,NaN,No injury,Social reason,Y,Y,...,Heading straight,NaN,NaN,M,1090,-123.343377,48.434454,2014-12-04 22:30:00,2014-12-04 22:30:00,2014-12-04
1,Collision with moving object or vehicle,"Vehicle, side",2015-07-28T10:30:00,collision,Vehicle turned left into my lane. I was unable...,NaN,"Injury, hospital emergency visit",Exercise or recreation,Y,Y,...,Heading straight,1941,5,M,1427,-123.418093,48.621903,2015-07-28 10:30:00,2015-07-28 10:30:00,2015-07-28
2,Collision with moving object or vehicle,"Vehicle, head on",2015-06-02T17:20:00,collision,I attended an head on accident between two mal...,NaN,"Injury, hospitalized",Exercise or recreation,NaN,NaN,...,Turning left,NaN,NaN,NaN,1510,-123.398636,48.592406,2015-06-02 17:20:00,2015-06-02 17:20:00,2015-06-02
3,Collision with moving object or vehicle,Another cyclist,2015-05-06T09:00:00,collision,I was traveling south on Borden crossing McKen...,NaN,"Injury, no treatment",Commute,Y,Y,...,Turning right,1958,7,M,917,-123.361423,48.470311,2015-05-06 09:00:00,2015-05-06 09:00:00,2015-05-06
4,Fall,Train Tracks,2015-05-30T17:30:00,collision,Riding up the hill and my wheel caught in the ...,NaN,"Injury, no treatment",Commute,Y,Y,...,Heading straight,1955,4,F,1052,-123.475707,48.447650,2015-05-30 17:30:00,2015-05-30 17:30:00,2015-05-30


In [198]:
#from shapely.geometry import Point
#collisions['geometry'] = collisions.apply(lambda x: Point((float(x.x), float(x.y))), axis=1)

#import geopandas
#gp_collisions = geopandas.GeoDataFrame(collisions, geometry='geometry')

#gp_collisions.head()

#gp_collisions.to_file('collisions_2.shp', driver='ESRI Shapefile')

In [217]:
collisions3 = collisions[['x', 'y', 'date']]

collisions3.head()

collisions3.to_csv('/Users/marthamorrissey/Documents/STA_project_MM/Data/xytime_collisions.csv')

In [218]:
path = "/anaconda/lib/python3.5/site-packages/pysal/examples/burkitt/burkitt.shp" 

#hacking work around- re-named my data to match the example data in PySAL examples folder 

In [220]:
events = interaction.SpaceTimeEvents(path, 'date')
events.n 

events.space[0]

events.t[0] #time not in the rigth formatt- must be in Python datetime 
events.t[1] - events.t[0]

#np.random.seed(100)

#result = interaction.modified_knox(events.space, events.t,delta=20,tau=5,permutations=99)

#print("%2.8f"%result['stat'])


TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U19') dtype('<U19') dtype('<U19')

In [215]:
b = pd.read_csv('/Users/marthamorrissey/Documents/pysal_example_burkitt.csv')

b.head()

b.dtypes

X         int64
Y         int64
ID      float64
X.1     float64
Y.1     float64
T       float64
AGE     float64
DATE     object
dtype: object

In [80]:
#path2 = "/anaconda/lib/python3.5/site-packages/pysal/examples/c2/c2.dbf"

In [11]:
#events = interaction.SpaceTimeEvents(path2, 'T')

In [10]:
#pysal.examples.get_path('c2')

In [90]:
import scipy
import treelib
import pyclust

In [ ]:
#pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)